In [1]:
#!pip install urllib3
#!pip install bs4

# For data manipulation
import pandas as pd
from urllib.request import urlopen, Request

# To extract fundamental data
from bs4 import BeautifulSoup

# Functions to Parse Data from FinViz

In [2]:
def fundamental_metric(soup, metric):
    return soup.find(text = metric).find_next(class_='snapshot-td2').text

In [3]:
def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
            req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
            response = urlopen(req)
            soup = BeautifulSoup(response)
            for m in df.columns:                
                df.loc[symbol,m] = fundamental_metric(soup,m)                
        except Exception as e:
            print (symbol, 'not found')
    return df

# List of Stocks and Ratios You are Interested In

In [4]:
stock_list = ['AMZN','GOOG','PG','KO','IBM','DG','XOM','KO','PEP','MT','NL','LPL']

metric = ['P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

# Initialize Pandas DataFrame to Store the Data

In [5]:
df = pd.DataFrame(index=stock_list,columns=metric)
df = get_fundamental_data(df)
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,11.25,47.22,42.12,1.36,0.48,64.71,-,28.00%,9.50%,96.40%,9.80%
GOOG,6.96,24.65,22.79,1.17,-,103.81,-,-,-,68.40%,0.04%
PG,8.66,27.76,24.90,4.39,0.81,5.66,2.22%,31.60%,18.30%,12.60%,0.11%
KO,12.11,28.63,24.51,3.98,1.86,2.25,2.61%,44.50%,12.60%,65.00%,0.10%
IBM,6.00,24.26,11.96,2.04,2.74,5.20,5.20%,27.20%,6.50%,-39.80%,0.10%
DG,9.23,24.61,20.03,2.27,0.00,10.17,0.67%,38.60%,24.50%,-1.90%,0.30%
XOM,2.25,16.47,12.20,1.03,0.28,5.38,3.98%,14.30%,7.20%,143.90%,0.10%
KO,12.11,28.63,24.51,3.98,1.86,2.25,2.61%,44.50%,12.60%,65.00%,0.10%
PEP,14.72,30.31,23.60,4.04,2.51,5.62,2.52%,49.80%,16.30%,-28.40%,0.10%
MT,0.66,2.43,5.02,-,0.17,13.67,0.90%,33.20%,25.20%,309.50%,44.80%


# Data Clearning: Further Parse the Data into Numeric Types
Remove % Sign and Convert Values to Numeric Type

In [6]:
df['Dividend %'] = df['Dividend %'].str.replace('%', '')
df['ROE'] = df['ROE'].str.replace('%', '')
df['ROI'] = df['ROI'].str.replace('%', '')
df['EPS Q/Q'] = df['EPS Q/Q'].str.replace('%', '')
df['Insider Own'] = df['Insider Own'].str.replace('%', '')
df = df.apply(pd.to_numeric, errors='coerce')
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,11.25,47.22,42.12,1.36,0.48,64.71,NaN,28.0,9.5,96.4,9.80
GOOG,6.96,24.65,22.79,1.17,NaN,103.81,NaN,NaN,NaN,68.4,0.04
PG,8.66,27.76,24.90,4.39,0.81,5.66,2.22,31.6,18.3,12.6,0.11
KO,12.11,28.63,24.51,3.98,1.86,2.25,2.61,44.5,12.6,65.0,0.10
IBM,6.00,24.26,11.96,2.04,2.74,5.20,5.20,27.2,6.5,-39.8,0.10
DG,9.23,24.61,20.03,2.27,0.00,10.17,0.67,38.6,24.5,-1.9,0.30
XOM,2.25,16.47,12.20,1.03,0.28,5.38,3.98,14.3,7.2,143.9,0.10
KO,12.11,28.63,24.51,3.98,1.86,2.25,2.61,44.5,12.6,65.0,0.10
PEP,14.72,30.31,23.60,4.04,2.51,5.62,2.52,49.8,16.3,-28.4,0.10
MT,0.66,2.43,5.02,NaN,0.17,13.67,0.90,33.2,25.2,309.5,44.80


# Filter Good Companies

### 1. Companies which are quoted at low valuations
P/E < 15 and P/B < 1

In [7]:
df_filtered = df[(df['P/E'].astype(float)<15) & (df['P/B'].astype(float) < 1)]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.66,2.43,5.02,NaN,0.17,13.67,0.90,33.2,25.2,309.5,44.8
NL,0.95,6.73,NaN,NaN,0.00,1.05,3.97,15.0,0.8,246.2,83.0
LPL,0.49,5.67,21.17,0.17,0.97,1.31,NaN,9.5,7.0,-68.8,51.1


### 2. Further filter companies which have demonstrated earning power 
EPS Q/Q > 10%

In [8]:
df_filtered = df_filtered[df_filtered['EPS Q/Q'].astype(float) > 10]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.66,2.43,5.02,NaN,0.17,13.67,0.90,33.2,25.2,309.5,44.8
NL,0.95,6.73,NaN,NaN,0.00,1.05,3.97,15.0,0.8,246.2,83.0


### Management having substantial ownership in the business
Insider Own > 30%

In [9]:
df = df[df['Insider Own'].astype(float) > 30]
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.66,2.43,5.02,NaN,0.17,13.67,0.90,33.2,25.2,309.5,44.8
NL,0.95,6.73,NaN,NaN,0.00,1.05,3.97,15.0,0.8,246.2,83.0
LPL,0.49,5.67,21.17,0.17,0.97,1.31,NaN,9.5,7.0,-68.8,51.1
